In [2]:
import requests
import pandas as pd
import numpy as np
import csv
import io
import datetime
from sqlalchemy import create_engine
import pyodbc
import urllib.request
import json
import pyodbc
from sqlalchemy import create_engine
import sqlalchemy
from pandas.io.json import json_normalize
import time
import dateparser
from googletrans import Translator
import timeit
import codecs
import pickle
import pythainlp

from pythainlp import sent_tokenize, word_tokenize
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from nltk.corpus import stopwords
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF

1. create dept grouping manually using IF contain text statement

2. Create topic grouping for project name
    2.1: Tokenize, remove stop word, remove unwanted word from project name
    2.2: Use tf idf to assign score for each word
    2.3: Use NMF to create 20 topic grouping.

In [3]:
df_contract = pd.read_csv("contract_eng_consolidated.csv")
df_project = pd.read_csv("project_eng_consolidated.csv")

C:\Users\v-krvibh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [151]:
df_project.shape

(401219, 16)

In [4]:


def dept_name_grouping(x):
    x = x.lower()
    if "hospital" in x:
        return "Hospital"
    elif any(ele in x for ele in ['university','campus','institute','college','faculty']):
        return "Higher education"
    elif any(ele in x for ele in ['school','kindergarten']):
        return 'School'
    elif any(ele in x for ele in ['municipality','subdistrict','sub-district','district','provincial']):
        return 'Local Government unit'
    elif any(ele in x for ele in ['office','department','government']):
        return 'department'
    elif 'provincial' in x:
        return 'provincial'
    elif 'police' in x:
        return 'police'
    return 'Other'

In [5]:
#df_project['dept_name_group'] = df_project['dept_name_eng'].apply(dept_name_grouping)

In [6]:
#df_project.to_csv("project_eng_consolidated.csv", index = False)

In [165]:
Topic_grouping = df_project[['project_name','dept_name_eng']]

In [166]:
#### Topic Modeling ########

## replace thai num with english num
# thai number in the list
num_map = {}
num_eng = 0
for num_thai in pythainlp.thai_digits:
    num_map[str(num_thai)] = str(num_eng)
    num_eng += 1
Topic_grouping['project_name'].replace(num_map, inplace=True, regex=True)

C:\Users\v-krvibh\anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [192]:
## function to remove stop word and unwanted word

## get word that have length greater than....
## get word that have count greater than...
def preprocess_data(txt_string):
    tokens = word_tokenize(txt_string, keep_whitespace=False)
    tokens = [i for i in tokens if not i in th_stop and not i in en_stop and not i in th_manual_word and len(i) > 2]
    return tokens

In [193]:
# put tokenize word back into list and join them using ,
def tokenize_text(raw_col):
    tokens_list = [preprocess_data(txt) for txt in raw_col]
    tokens_list_j = [','.join(tkn) for tkn in tokens_list]
    return tokens_list_j

In [194]:
en_stop = set(stopwords.words('english'))
th_stop = pythainlp.corpus.thai_stopwords()
th_manual_word = ['ประกวดราคา', 'จ้าง','ทางอิเล็กทรอนิกส์','ซื้อ','กรณีพิเศษ','(',')' ,'/' ,'+','-' ,'e-bidding',
                  'อิเล็กทรอนิกส์']

project_type_name_th = ['จ้างก่อสร้าง', 'จ้างทำของ/จ้างเหมาบริการ', 'ซื้อ', 'เช่า',
       'จ้างที่ปรึกษา', 'จ้างควบคุมงาน', 'จ้างออกแบบ',
       'จ้างออกแบบและควบคุมงาน']

purchase_method_group_name_th = ['ประกวดราคา', 'ประกวดราคาด้วยวิธีการทางอิเล็กทรอนิกส์', 'พิเศษ',
       'จ้างที่ปรึกษา', 'กรณีพิเศษ',
       'ประกวดราคาอิเล็กทรอนิกส์ (e-bidding)', 'สอบราคา', 'ตกลงราคา',
       'ตลาดอิเล็กทรอนิกส์ (e-market)', 'จ้างออกแบบ', 'คัดเลือก',
       'เฉพาะเจาะจง']

purchase_method_group_name_eng = ['Bidding', 'E-bidding', 'Special',
        'Hire consultants', 'Special cases',
        'E-bidding','Price examination','Agreed price',
        'E-market', 'Hire to design', 'selection',
        'Specific']

province_th = ['นนทบุรี', 'กรุงเทพมหานคร', 'อุตรดิตถ์', 'เชียงราย', 'ชลบุรี',
       'พระนครศรีอยุธยา', 'สมุทรปราการ', 'ปทุมธานี', 'ลำปาง', 'นครนายก',
       'เชียงใหม่', 'ขอนแก่น', 'พะเยา', 'ปัตตานี', 'ลพบุรี',
       'อุบลราชธานี', 'ยะลา', 'นครราชสีมา', 'อุดรธานี', 'สมุทรสาคร',
       'บึงกาฬ', 'จันทบุรี', 'ฉะเชิงเทรา', 'นครศรีธรรมราช', 'สงขลา',
       'นครปฐม', 'มหาสารคาม', 'กาญจนบุรี', 'แพร่', 'พิจิตร', 'ภูเก็ต',
       'พิษณุโลก', 'กาฬสินธุ์', 'กำแพงเพชร', 'สุราษฎร์ธานี', 'ระยอง',
       'ศรีสะเกษ', 'เพชรบุรี', 'เลย', 'ชัยภูมิ', 'สกลนคร', 'นครพนม',
       'บุรีรัมย์', 'สระบุรี', 'ปราจีนบุรี', 'สุรินทร์', 'นครสวรรค์',
       'เพชรบูรณ์', 'นราธิวาส', 'ประจวบคีรีขันธ์', 'ลำพูน', 'ยโสธร',
       'สุโขทัย', 'สตูล', 'พังงา', 'ชัยนาท', 'กระบี่', 'พัทลุง', 'ชุมพร',
       'ตาก', 'ตรัง', 'อำนาจเจริญ', 'ราชบุรี', 'ร้อยเอ็ด', 'หนองคาย',
       'มุกดาหาร', 'สิงห์บุรี', 'สมุทรสงคราม', 'สุพรรณบุรี', 'อ่างทอง',
       'ตราด', 'ระนอง', 'น่าน', 'อุทัยธานี', 'สระแก้ว', 'หนองบัวลำภู',
       'แม่ฮ่องสอน']

th_manual_word = th_manual_word + project_type_name_th + purchase_method_group_name_th + purchase_method_group_name_eng +province_th

In [196]:
# create a column of tokenize word
Topic_grouping['clean_project_name'] = tokenize_text(Topic_grouping['project_name'])

C:\Users\v-krvibh\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [197]:

#Tfidf: Count how often it occur in each document as well as how often it occur overall
# if occur a lot in that document, the word is important, but if occur a lot in overall, not as important
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
X = tvec.fit_transform(Topic_grouping['clean_project_name'])

In [200]:
######## Method 3: LDA & NMF ##########


#helper method to print top 10 topics
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
#####
tf_feature_names = tvec.get_feature_names()

In [203]:
#### NMF
nmf = NMF(n_components=20, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(X)

In [228]:
print_top_words(nmf, tf_feature_names, 50)

Topic #0: อำเภอ จังหวัด ตำบล ก่อสร้าง หมู่ บ้าน วิธี โครงการ เมือง สาย ถนน ท่า ห้วย หนอง วัง คลอง วิธีการ หิน ทอง ทุ่ง คอนกรีตเสริมเหล็ก โครง ยาง แม่ ถนนลาดยาง ป่า ศรี เกาะ น้ำ วัด ผิวจราจร การปรับปรุง แก้ว ฝาย เชื่อม ช้าง ปาก ทราย บึง บ่อ ระยะทาง ชัย ควน ไทร เชื่อมต่อ หลวง โคก บุรี หัว พระ
Topic #1: รายการ จำนวน วิธี ครุภัณฑ์ การแพทย์ น้ำมันเชื้อเพลิง เวชภัณฑ์ วัสดุก่อสร้าง (ย. ทางการแพทย์ โรงพยาบาล tablet ประเภท วิทยาศาสตร์ 100 000 น้ำมันดีเซลหมุนเร็ว อากาศยาน ลิตร น้ำมัน 51.99.99.99 injection ปอร์ตแลนด์ พัสดุ ปูนซีเมนต์ กล่อง ผลิต สำหรับ solution มหาราช มอก. ขาย กก. 500 พิมพ์ 12.18.16.00 งานบ้าน เม็ด เลขที่ film-coated ครัว 1267 2544 ผลิตภัณฑ์ capsule ราคา 250 ASL รอง vial
Topic #2: กม. ตอน ทางหลวง งาน หมายเลข ปริมาณ ควบคุม เหมา ทำการ จ้างงาน แอสฟัลต์ ผิว 000 ตร. บูรณะ กิจกรรม 0100 อันตราย 500 ราว  ต้น ทำงาน เสริม การงาน ติดตั้ง บำรุงรักษา 0102 เครื่องหมายจราจร แขวง แสงสว่าง เป็นช่วงๆ ป้ายจราจร 800 200 รหัส 0101 700 900 ระยะทาง ฉาบ 600 300 ซ่อม ห้วย แม่ เป็นช่วง ซ้าย สะพาน มาตรฐาน
T

Factor A into W,H

Use W to get topics

In [229]:
W = nmf.fit_transform(X)

In [230]:
assign_topic = np.argmax(W,axis = 1)

In [236]:
for i in range(nmf.n_components):
    print("Number of documents in topics ", i," is ", np.sum(assign_topic==i), " and ", np.sum(assign_topic==i)/len(assign_topic) *100, '% of total ')

Number of documents in topics  0  is  35097  and  8.74759171425082 % of total 
Number of documents in topics  1  is  31119  and  7.756113244886209 % of total 
Number of documents in topics  2  is  23834  and  5.940396641235834 % of total 
Number of documents in topics  3  is  26674  and  6.648239490153757 % of total 
Number of documents in topics  4  is  19092  and  4.758498475894711 % of total 
Number of documents in topics  5  is  12552  and  3.1284659998654103 % of total 
Number of documents in topics  6  is  19699  and  4.909787422828929 % of total 
Number of documents in topics  7  is  22893  and  5.705861387421832 % of total 
Number of documents in topics  8  is  20875  and  5.202894179986491 % of total 
Number of documents in topics  9  is  17111  and  4.264753164730484 % of total 
Number of documents in topics  10  is  18563  and  4.62665028326176 % of total 
Number of documents in topics  11  is  13663  and  3.4053721284385836 % of total 
Number of documents in topics  12  is 

In [ ]:
### find location where 
### 
'''
feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
                             
1. return index of the array in order to sort it from 0 to max
2. 

    '''


In [242]:
df_project["Topic_group"] = assign_topic

In [246]:
#df_project.to_csv('project_eng_consolidated_with_Topics.csv',index=False)

In [247]:
df_project.head()

,project_type_name_eng,purchase_method_group_name_eng,province_eng,project_status_eng,district_eng,announce_date_eng,transaction_date_eng,project_money_cleaned,price_build_cleaned,sum_price_agree_cleaned,dept_name_eng,budget_year_cleaned,project_id,project_name,geom,dept_name_group,Topic_group
0,Hire for construction,Bidding,Nonthaburi,In progress,Bang Kruai,2 Apr 2013,9 March 2015,0,0,70624000000,Electricity Generating Authority of Thailand,2015,56045006726,ประกวดราคาจ้างซื้อและจ้างก่อสร้างโรงไฟฟ้าเพื่อ...,POINT(100.498178770987 13.8068549692447),Other,4
1,Outsourcing / Contracting services,Bidding,Nonthaburi,In progress,Bang Kruai,21 Nov 2014,26 Aug 2015,30000000000,26910000000,22871000000,Electricity Generating Authority of Thailand,2015,57115200071,ประกวดราคาจ้างเหมาขุด-ขนดินและถ่าน ที่เหมืองแม...,POINT(100.498178770987 13.8068549692447),Other,14
2,Outsourcing / Contracting services,E-bidding,Bangkok,In progress,Din,27 Aug 2014,13 Oct 2014,4455190000,4480010000,4436575000,Bangkok,2015,57085094977,ประกวดราคาจ้างเหมาเอกชนขนมูลฝอยจากศูนย์กำจัดมู...,POINT(100.565063509801 13.7780980259196),Other,14
3,buy,Special,Bangkok,In progress,Dusit,NaN,28 Jan 2015,2800000000,2799999972,2725935375,Army,2015,57085194402,ซื้อฮ.ท.ขนาดเบา แบบที่ 2 จำนวน 6 เครื่อง โดยวิ...,POINT(100.519873059321 13.7873113160748),Other,6
4,Hire for construction,E-bidding,Bangkok,In progress,Chatuchak,25 Mar 2014,9 Oct 2014,2599999769,2599999769,2588950000,Army,2015,57035180671,ประกวดราคาจ้างก่อสร้างอาคารเฉลิมพระเกียรติ สมเ...,POINT(100.584152190858 13.8453156541146),Other,3
